<img src="http://oproject.org/tiki-download_file.php?fileId=8&display&x=450&y=128">
<img src="http://gfif.udea.edu.co/root/tmva/img/tmva_logo.gif">
## TMVA Deep Neural Network Example

In [1]:
TMVA::Tools::Instance();

### Open input and output files

In [2]:
TString infilename = "higgs.root";
TFile *input       = TFile::Open(infilename);
TMVA::DataLoader *loader = new TMVA::DataLoader("higgs");

TString outfilename = "TMVA_DNN_GPU_1.root";
TFile *output       = TFile::Open(outfilename, "RECREATE");

TMVA::Factory *factory   = new TMVA::Factory("TMVAClassification",
                                             output,
                                             "AnalysisType=Classification");

--- Factory                  : You are running ROOT Version: 6.07/07, Apr 1, 2016
--- Factory                  : 
--- Factory                  : _/_/_/_/_/ _|      _|  _|      _|    _|_|   
--- Factory                  :    _/      _|_|  _|_|  _|      _|  _|    _| 
--- Factory                  :   _/       _|  _|  _|  _|      _|  _|_|_|_| 
--- Factory                  :  _/        _|      _|    _|  _|    _|    _| 
--- Factory                  : _/         _|      _|      _|      _|    _| 
--- Factory                  : 
--- Factory                  : ___________TMVA Version 4.2.1, Feb 5, 2015
--- Factory                  : 


### Prepare training data

In [3]:
TTree *tsignal     = (TTree *) input->Get("TreeS");
TTree *tbackground = (TTree *) input->Get("TreeB");
Double_t signalWeight = 1.0;
Double_t backgroundWeight = 1.0;
loader->AddSignalTree    (tsignal,     signalWeight);
loader->AddBackgroundTree(tbackground, backgroundWeight);

--- DataSetInfo              : Dataset[higgs] : Added class "Signal"	 with internal class number 0
--- higgs                    : Add Tree TreeS of type Signal with 5829123 events
--- DataSetInfo              : Dataset[higgs] : Added class "Background"	 with internal class number 1
--- higgs                    : Add Tree TreeB of type Background with 5170877 events


### Add variables

In [4]:
loader->AddVariable("lepton_pT",'F');
loader->AddVariable("lepton_eta",'F');
loader->AddVariable("lepton_phi",'F');
loader->AddVariable("missing_energy_magnitude",'F');
loader->AddVariable("missing_energy_phi",'F');
loader->AddVariable("jet_1_pt",'F');
loader->AddVariable("jet_1_eta",'F');
loader->AddVariable("jet_1_phi",'F');
loader->AddVariable("jet_1_b_tag",'F');
loader->AddVariable("jet_2_pt",'F');
loader->AddVariable("jet_2_eta",'F');
loader->AddVariable("jet_2_phi",'F');
loader->AddVariable("jet_2_b_tag",'F');
loader->AddVariable("jet_3_pt",'F');
loader->AddVariable("jet_3_eta",'F');
loader->AddVariable("jet_3_phi",'F');
loader->AddVariable("jet_3_b_tag",'F');
loader->AddVariable("jet_4_pt",'F');
loader->AddVariable("jet_4_eta",'F');
loader->AddVariable("jet_4_phi",'F');
loader->AddVariable("jet_4_b_tag",'F');
loader->AddVariable("m_jj",'F');
loader->AddVariable("m_jjj",'F');
loader->AddVariable("m_lv",'F');
loader->AddVariable("m_jlv",'F');
loader->AddVariable("m_bb",'F');
loader->AddVariable("m_wbb",'F');
loader->AddVariable("m_wwbb",'F');

TString dataString = "nTrain_Signal=5000000:"
                     "nTrain_Background=5000000:"
                     "nTest_Signal=10000:"
                     "nTest_Background=10000:"
                     "SplitMode=Random:"
                     "NormMode=NumEvents:"
                     "!V";
loader->PrepareTrainingAndTestTree("", "", dataString);


--- higgs                    : Preparing trees for training and testing...


### Define network structure and training scheme

In [5]:
TString configString = "!H:V";
configString += ":Architecture=GPU";
configString += ":VarTransform=N";
configString += ":ErrorStrategy=CROSSENTROPY";
configString += ":WeightInitialization=XAVIERUNIFORM";
TString layoutString = "Layout=RELU|200,RELU|200,RELU|200,LINEAR";

TString trainingString = "TrainingStrategy="
                         "LearningRate=0.1,"
                         "Momentum=0.5,"
                         "Repetitions=1,"
                         "ConvergenceSteps=5,"
                         "BatchSize=200,"
                         "DropConfig=1.0+0.5+0.5+1.0"
                         "Regularization=L2,"
                         "WeightDecay=1e-4,"
                         "TestRepetitions=2,"
                         "Multithreading=True";

configString += ":" + layoutString + ":" + trainingString;
TMVA::Types::EMVA method = TMVA::Types::kDNN;
factory->BookMethod(loader, method, "DNN", configString);

--- Factory                  : Booking method: DNN DataSet Name: higgs
--- <VERBOSE> Configurable             : Parsing option string: 
--- <VERBOSE> Configurable             : ... "!H:V:Architecture=GPU:VarTransform=N:ErrorStrategy=CROSSENTROPY:WeightInitialization=XAVIERUNIFORM:Layout=RELU|200,RELU|200,RELU|200,LINEAR:TrainingStrategy=LearningRate=0.1,Momentum=0.5,Repetitions=1,ConvergenceSteps=5,BatchSize=200,DropConfig=1.0+0.5+0.5+1.0Regularization=L2,WeightDecay=1e-4,TestRepetitions=2,Multithreading=True"
--- <VERBOSE> Configurable             : The following options are set:
--- <VERBOSE> Configurable             : - By User:
--- <VERBOSE> Configurable             :     <none>
--- <VERBOSE> Configurable             : - Default:
--- <VERBOSE> Configurable             :     Boost_num: "0" [Number of times the classifier will be boosted]
--- <VERBOSE> DNN                      : Parsing option string: 
--- <VERBOSE> DNN                      : ... "!H:V:Architecture=GPU:VarTransform=N

### Train and evaluate method

In [6]:
factory->TrainAllMethods();
factory->TestAllMethods();
factory->EvaluateAllMethods();

--- Factory                  :  
--- Factory                  : Train all methods for Classification ...
--- DataSetFactory           : Dataset[higgs] : Splitmode is: "RANDOM" the mixmode is: "SAMEASSPLITMODE"
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Signal" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Signal': ""
--- DataSetFactory           : Dataset[higgs] : Create training and testing trees -- looping over class "Background" ...
--- DataSetFactory           : Dataset[higgs] : Weight expression for class 'Background': ""
--- DataSetFactory           : Dataset[higgs] : Number of events in input trees (after possible flattening of arrays):
--- DataSetFactory           : Dataset[higgs] :     Signal          -- number of events       : 5829123  / sum of weights: 5.82912e+06
--- DataSetFactory           : Dataset[higgs] :     Background      -- number of events       : 5170877  / sum of we

## Plot results

## %jsroot
TCanvas *c=factory->GetROCCurve(loader);
c->Draw();

In [7]:
%jsroot
TCanvas *d=factory->GetROCCurve(loader);
d->Draw();


In [8]:
printf("%f \n", factory->GetROCIntegral(loader, "DNN"));

0.500000 
